<a href="https://colab.research.google.com/github/harithayallupu2005/Fmml-lab-1/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [ ]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-50-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [ ]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [ ]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [ ]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

# **QUESTION-1:**
# **How is the accuracy of the validation set affected if we increase the percentage of validation set?What happens when we reduce it?**

**Ans:** The accuracy of the validation set will generally increase as the percentage of validation set increases.This is because the model will have less data to overfit to,and will therefore be more generalizable to new data.However,if the percentage of validation set becomes to large,the model may not have enough data to learn from,and its accuracy may start to decrease.On the other hand,if we reduce the percentage of the validation set,the accuracy might decrease because the model has less data to validate its performance.



In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
x,y = np.random.rand(100,10),np.random.randint(0,2,100)
validation_percentages = [0.1,0.2,0.3,0.4,0.5]
for val_percentage in validation_percentages:
  x_train,x_val,y_train,y_val = train_test_split(x,y,test_size=val_percentage,random_state=42)
  model= LogisticRegression()
  model.fit(x_train,y_train)
  y_pred=model.predict(x_val)
  accuracy = accuracy_score(y_val, y_pred)
  print(f"validation Set Percentage: {val_percentage * 100}% | Accuracy: {accuracy * 100:.2f}%")

validation Set Percentage: 10.0% | Accuracy: 50.00%
validation Set Percentage: 20.0% | Accuracy: 45.00%
validation Set Percentage: 30.0% | Accuracy: 40.00%
validation Set Percentage: 40.0% | Accuracy: 45.00%
validation Set Percentage: 50.0% | Accuracy: 42.00%


# **QUESTION-2:**
# **How does the size of the train and validation set affect how well we can predict the accuracy on the test using the validation set?**

**Ans:** The size of the train and validation sets can affect how well we can predict the accuracy on the test using the validation set.If we have a larger training set,the model can learn more patterns and potentially perform better on the test set. However,if the validation set is too small,it may not accurately represent the test set,leading to less reliable predictions of accuracy. It's important to find the right balance between the sizes of the train and validation sets for accurate predictions.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
x,y=np.random.rand(100,10),np.random.randint(0,2,100)
train_sizes=[0.2,0.4,0.6,0.8]
for train_size in train_sizes:
  x_train,x_val,y_train,y_val=train_test_split(x,y,train_size=train_size,random_state=42)
  model = LogisticRegression()
  model.fit(x_train,y_train)
  y_pred=model.predict(x_val)
  accuracy = accuracy_score(y_val,y_pred)
  print(f"Training Set Size: {train_size * 100}% | Accuracy: {accuracy * 100:2f}%")

Training Set Size: 20.0% | Accuracy: 40.000000%
Training Set Size: 40.0% | Accuracy: 33.333333%
Training Set Size: 60.0% | Accuracy: 40.000000%
Training Set Size: 80.0% | Accuracy: 60.000000%


# **QUESTION-3:**
# **What do you think is a good percentage to reserve for the validation set so that these two factors are balanced?**

**Ans:** It's hard to give an exact percentage since it can vary depending on the specific dataset and problem.But a common practice is to reserve around 20-30% of the data for the validation data set.This allows for a sufficient amount of data for training while also providing a sizable portion for validation.It's always a good idea to experiment with different percentages to find the best balance for your specific scenario.

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
x,y = np.random.rand(100,10),np.random.randint(0,2,100)
validation_percentages = [0.1,0.15,0.2,0.25,0.3]
best_accuracy=0
best_val_percentage=0
for val_percentage in validation_percentages:
  x_train,x_val,y_train,y_val = train_test_split(x,y,test_size=val_percentage,random_state=42)
  model= LogisticRegression()
  model.fit(x_train,y_train)
  y_pred=model.predict(x_val)
  accuracy = accuracy_score(y_val, y_pred)
  if accuracy > best_accuracy:
    best_accuracy=accuracy
    best_val_percentage=val_percentage
  print(f"Best Validation Set Percentage: {best_val_percentage * 100}% | Best_Accuracy: {best_accuracy * 100:.2f}%")

Best Validation Set Percentage: 10.0% | Best_Accuracy: 40.00%
Best Validation Set Percentage: 15.0% | Best_Accuracy: 46.67%
Best Validation Set Percentage: 20.0% | Best_Accuracy: 55.00%
Best Validation Set Percentage: 25.0% | Best_Accuracy: 56.00%
Best Validation Set Percentage: 25.0% | Best_Accuracy: 56.00%


## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [ ]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [ ]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


# **QUESTION-1:**

**Does averaging the validation accuracy across multiple splits give more consistent results?**

**Ans:** YES.

Averaging the validation accuracy across multiple splits can give more consistent results. By performing multiple splits of the data into train and validation sets and calculating the accuracy for each split,we can reduce the impact of randomness and obtain a more reliable estimate of the model's performance. It helps to get a better understanding of how the model generalizes to different subsets of the data.

In [ ]:
import numpy as h
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
x,y=h.random.rand(100,10),h.random.randint(0,2,100)
model=LogisticRegression()
k=5
accuracies= cross_val_score(model,x,y,cv=k)
avg_accuracy=h.mean(accuracies)
std_dev=h.std(accuracies)
print(f"Average Accuracy: {avg_accuracy * 100:2f}%")
print(f"Standard Deviation: {std_dev * 100:2f}%")

Average Accuracy: 46.000000%
Standard Deviation: 8.602325%


# **QUESTION-2:**

**Does it give more accurate estimate of test accuracy?**

**Ans:** YES.

Averaging the validation accuracy across multiple splits can provide a more accurate estimate of the test accuracy. By evaluating the model's performance on different subsets of the data, we can get a better sense of its generalization ability. This helps to reduce the impact of any specific split and provides a more reliable estimation of how the model will perform on unseen data.

In [ ]:
import numpy as h
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
x,y=h.random.rand(100,10),h.random.randint(0,2,100)
param_grid={'C': [0.001,0.01,0.1,1,10]}
model=LogisticRegression()
n_splits=5
stratified_kfold=StratifiedKFold(n_splits=n_splits,shuffle=True,random_state=42)
test_accuracies=cross_val_score(model,x,y,cv=stratified_kfold)
avg_test_accuracy=h.mean(test_accuracies)
std_dev_test_accuracy=h.std(test_accuracies)
print(f"Average Test Accuracy: {avg_test_accuracy * 100:2f}%")
print(f"Standard Deviation of Test Accuracy: {std_dev_test_accuracy * 100:2f}%")

Average Test Accuracy: 45.000000%
Standard Deviation of Test Accuracy: 3.162278%


# **QUESTION-3:**

# **What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?**

**Ans:** With higher iterations,we might get a better estimate of the model's performance.More iterations allow the model to learn from the data in greater detail and make finer adjustments to improve its accuracy. However,there is a point of diminshing returns,where increasing iterations may not significantly improve the estimate and may even lead to overfitting. It's important to find the right balance between the number of iterations and the model's performance.

In [ ]:
import random
def estimate_average(iterations):
  total_sum=0
  for i in range(iterations):
      random_number=random.uniform(0,1)
      total_sum +=random_number
  average_estimate=total_sum/iterations
  return average_estimate
h=[100,1000,10000,100000,1000000]
for iterations in h:
  avg_estimate = estimate_average(iterations)
  print(f"Iterations: {iterations},Estimated Average: {avg_estimate}")


Iterations: 100,Estimated Average: 0.4896658088222076
Iterations: 1000,Estimated Average: 0.5005167391697541
Iterations: 10000,Estimated Average: 0.5017757121197657
Iterations: 100000,Estimated Average: 0.5003442606347793
Iterations: 1000000,Estimated Average: 0.4996840308018541


# **QUESTION-4:**

# **Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?**

**Ans:** YES.

Increasing the iterations can help mitigate the impact of a small train or validation dataset to some extent.By allowing the model to go through more iterations,it gets more opportunities to learn from the available data make better predictions. However,it's important to note that while increasing iterations can compensate for small datasets to some degree,it is still crucial to have a sufficiently sized dataset for accurate model training and validation.

In [ ]:
import numpy as h
def generate_random_data(num_points):
  x=h.random.rand(num_points)
  y = 2 * x + 1 + h.random.randn(num_points) * 0.1
  return x,y
iterations=10
x_train,y_train=generate_random_data(iterations)
for i in range(iterations):
  print(f"Data Point {i+1}: X={x_train[i]},Y={y_train[i]}")

Data Point 1: X=0.5573046545917625,Y=2.0557284788587595
Data Point 2: X=0.4324522433164001,Y=1.9284694274846264
Data Point 3: X=0.2978077610161257,Y=1.5171663352802514
Data Point 4: X=0.1121096074306025,Y=1.318516102218591
Data Point 5: X=0.1611347101318391,Y=1.1791894117950947
Data Point 6: X=0.8294574969253435,Y=2.8700108584269906
Data Point 7: X=0.1425566913993449,Y=1.2589508356682517
Data Point 8: X=0.8172982793951008,Y=2.6621503750454787
Data Point 9: X=0.6946953285561458,Y=2.2534863894660804
Data Point 10: X=0.45978078674288725,Y=1.9608262132489578
